<a href="https://colab.research.google.com/github/A23929/Machinehack-/blob/main/shell_hackathon_to_protect_against_cyber_threats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [11]:
df = pd.read_json("datafinal.json")
df

,ID,Text,ContainsCode,CodeList
0,1,Test-driven development (TDD) is a software de...,,
1,2,Software development is an exciting field that...,False,
2,3,The development process often starts with a co...,,
3,4,Another important aspect of software developme...,True,git commit -m 'Initial commit'
4,5,"In the world of software development, language...",False,
...,...,...,...,...
1903,1904,"Software development, an integral part of any ...",True,List<String> names = new ArrayList<String>();
1904,1905,"In the world of software development, one of t...",False,
1905,1906,Web development is a popular field in software...,,
1906,1907,C++ is a general-purpose programming language ...,,


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908 entries, 0 to 1907
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            1908 non-null   int64 
 1   Text          1908 non-null   object
 2   ContainsCode  1908 non-null   object
 3   CodeList      1908 non-null   object
dtypes: int64(1), object(3)
memory usage: 74.5+ KB


### Write code to process the data and populate each row with the respective code. Keep in mind that this is not a machine learning problem; you can use any software engineering approach to fill the rows with code

In [13]:
df["CodeList"].fillna("",inplace=True) ## Incase if it contain any NaN values

In [14]:
for x in range(len(df['ContainsCode'])):
  if df['ContainsCode'][x] == '':
    df['ContainsCode'][x] = False

<ipython-input-14-cc0b9ee33659>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ContainsCode'][x] = False


In [15]:
df['ContainsCode'] = df['ContainsCode'].astype(int)

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
s1 = df['CodeList']
t1 = mlb.fit_transform(s1)

In [17]:
data = pd.DataFrame(t1)

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import re
from nltk.tokenize import word_tokenize

# Function to clean and tokenize text
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the Text column
df['Text'] = df['Text'].apply(preprocess_text)

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['ContainsCode'], test_size=0.2, random_state=42)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


## Sec 1

In [22]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

In [37]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.7094240837696335
              precision    recall  f1-score   support

           0       0.74      0.91      0.82       275
           1       0.45      0.19      0.26       107

    accuracy                           0.71       382
   macro avg       0.60      0.55      0.54       382
weighted avg       0.66      0.71      0.66       382



## Sec 2

In [41]:
import xgboost as xgb

model = xgb.XGBClassifier(subsample = 0.5)
model.fit(X_train_tfidf, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [43]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.7172774869109948
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       275
           1       0.49      0.36      0.42       107

    accuracy                           0.72       382
   macro avg       0.63      0.61      0.62       382
weighted avg       0.70      0.72      0.70       382



## Make sure the matrix of submission file is always 1908 rows × 93 columns

In [47]:
submissions_df = df

In [48]:
# Load the submission data
submissions_df['Text'] = df['Text'].apply(preprocess_text)

# Vectorize the submission data
X_submission_tfidf = tfidf_vectorizer.transform(submissions_df['Text'])

# Predict
submission_predictions = model.predict(X_submission_tfidf)

# Add the predictions to the submissions DataFrame
submissions_df['ContainsCode'] = submission_predictions

mlb = MultiLabelBinarizer()
s1 = submissions_df['CodeList']
t1 = mlb.fit_transform(s1)
submissions_df = pd.DataFrame(t1)

# Save the results to a new CSV file
submissions_df.to_csv("submission_results.csv", index=False)


In [49]:
submissions_df.shape

(1908, 93)